In [1]:
import os 
%pwd


'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/viyas52/End-To-End-Wine-Quality-Pred.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="viyas52"
os.environ["MLFLOW_TRACKING_PASSWORD"]="396092e9fb18dff075743b0080074533456e33c1"
